# Generative Search with Mistral

In this demo, we will use Mistral to generate the embeddings for the blog post and use Mistral's language model to generate content!

What you will need for this demo:
1. A Weaviate cluster (more info below)
2. Mistral API key

In [2]:
import weaviate
from weaviate.embedded import EmbeddedOptions
import weaviate.classes as wvc
import weaviate.classes.config as wc
import requests, json
import weaviate.classes.query as wq
from weaviate.classes.config import Property, DataType
import os
import re
from weaviate.util import get_valid_uuid
from uuid import uuid4

## Connect to Weaviate

Only choose one option from the below.

**Weaviate Cloud Deployment**

In [ ]:
WCD_URL = os.environ["WEAVIATE_URL"] # Replace with your Weaviate cluster URL
WCD_AUTH_KEY = os.environ["WEAVIATE_AUTH"] # Replace with your cluster auth key
MISTRAL_KEY = os.environ["MISTRAL_API_KEY"] # Replace with your Mistral key

# Weaviate Cloud Deployment
client = weaviate.connect_to_wcs(
    cluster_url=WCD_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WCD_AUTH_KEY),
      headers={ "X-Mistral-Api-Key": MISTRAL_KEY}
)

print(client.is_ready())

**Embedded Weaviate**

In [ ]:
# MISTRAL_KEY = os.environ["MISTRAL_API_KEY"] # Replace with your Mistral key

# client = weaviate.WeaviateClient(
#     embedded_options=EmbeddedOptions(
#         version="1.26.1",
#         additional_env_vars={
#             "ENABLE_MODULES": "text2vec-mistral, generative-mistral"
#         }),
#         additional_headers={
#             "X-Mistral-Api-Key": MISTRAL_KEY
#         }
# )

# client.connect()

**Local Deployment**

In [ ]:
# MISTRAL_KEY = os.environ["MISTRAL_API_KEY"] # Replace with your Mistral key

# client = weaviate.connect_to_local(
#   headers={
#     "X-Mistral-Api-Key": MISTRAL_KEY
#   }
# )
# print(client.is_ready())

## Create a collection
Collection stores your data and vector embeddings.

Full list of [generative models](https://weaviate.io/developers/weaviate/model-providers/octoai/generative#available-models)

In [ ]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "JeopardyQuestion", and then you need to re-import it again.

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="BlogChunks",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_mistral( # specify the vectorizer and model
        model="mistral-embed",
    ),

    generative_config=wc.Configure.Generative.mistral( # specify the generarive model
        model="open-mistral-7b"
    ),

    properties=[
            Property(name="content", data_type=DataType.TEXT) # We only have one property for our collection. It is the content within the blog posts
    ]
)

print("Successfully created collection: BlogChunks.")

## Chunk and Import Data

We need to break our blog posts into smaller chunks

In [ ]:
def chunk_list(lst, chunk_size):
    """Break a list into chunks of the specified size."""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

def split_into_sentences(text):
    """Split text into sentences using regular expressions."""
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

def read_and_chunk_index_files(main_folder_path):
    """Read index.md files from subfolders, split into sentences, and chunk every 5 sentences."""
    blog_chunks = []
    
    for file_path in os.listdir("./data"):
        index_file_path = os.path.join("./data", file_path)
        with open(index_file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            sentences = split_into_sentences(content)
            sentence_chunks = chunk_list(sentences, 5)
            sentence_chunks = [' '.join(chunk) for chunk in sentence_chunks]
            blog_chunks.extend(sentence_chunks)
    return blog_chunks

# run with:
main_folder_path = './data'
blog_chunks = read_and_chunk_index_files(main_folder_path)

In [ ]:
# First chunk

blog_chunks[0]

In [ ]:
# Insert the objects (chunks) into the Weaviate cluster

blogs = client.collections.get("BlogChunks")

for blog_chunk in blog_chunks:
    random_uuid = get_valid_uuid(uuid4())
    blogs.data.insert(
        properties={
            "content": blog_chunk
        },
        uuid=random_uuid
    )


## Query Time 

## Hybrid Search Query

Hybrid search combines BM25 and vector search and weighs the two algorithms depending on the `alpha` parameter. 

`alpha`= 0 --> pure BM25

`alpha`= 0.5 --> half BM25, half vector search

`alpha`= 1 --> pure vector search

In [ ]:
import json 

blogs = client.collections.get("BlogChunks")

response = blogs.query.hybrid(
    query="What is Ref2Vec",
    alpha=0.5,
    limit=3
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

### Generative Search Query

Here is what happens in the below:
1. We will retrieve 3 relevant chunks from our vector database
2. We will pass the 3 chunks to Mistral to generate the short paragraph about Ref2Vec

The first line in the output is the generated text, and the `content` pieces below it, are what was retrieved from Weaviate and passed to Mistral.

In [ ]:
blogs = client.collections.get("BlogChunks")


response = blogs.generate.near_text(
    query="What is Ref2Vec?",
    single_prompt="Write a short paragraph about ref2vec with this content: {content}",
    limit=3
)


for o in response.objects:
    print(o.generated)
    print(json.dumps(o.properties, indent=2))